covid_chatbot

préparé par: Kheira , Nour

Préparation des données textuelles

In [4]:
import pandas as pd
import matplotlib.pyplot as plt

# Charger un fichier CSV
df = pd.read_csv("/content/covid_faq.csv")
df

,questions,answers
0,What is a novel coronavirus?,A novel coronavirus is a new coronavirus that ...
1,Why is the disease being called coronavirus di...,"On February 11, 2020 the World Health Organiza..."
2,How does the virus spread?,The virus that causes COVID-19 is thought to s...
3,Can I get COVID-19 from food (including restau...,Currently there is no evidence that people can...
4,Will warm weather stop the outbreak of COVID-19?,It is not yet known whether weather and temper...
...,...,...
112,Can the sun’s ultraviolet (UV) light disinfect...,We don’t know yet. Early scientific data sugge...
113,How often do surfaces (such as handrails and i...,The more frequently a surface is touched by mu...
114,Do procedures for responding to formed fecal (...,No. RNAexternal icon of the virus that causes ...
115,Can lifeguards monitor how visitors socially d...,Not when lifeguards are actively lifeguarding....


si il ya des valeurs manquantes

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   questions  117 non-null    object
 1   answers    117 non-null    object
dtypes: object(2)
memory usage: 2.0+ KB


Nettoyage des Données Textuelles

In [6]:
import re

# Fonction de nettoyage
def clean_text(text):
    text = text.lower()  # Conversion en minuscules
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Supprimer tout sauf lettres et espaces
    text = re.sub(r'\s+', ' ', text)  # Remplace les espaces multiples par un seul
    return text.strip()  # Supprime les espaces en début et en fin

# Vérifier les noms de colonnes
print(df.columns)  # Afficher les noms de colonnes disponibles

# Assuming 'questions' is the correct column name, modify the following line:
# Remplacer 'question' par le nom correct de la colonne
df['Cleaned_Question'] = df['questions'].apply(clean_text)

# Vérification après nettoyage
df[['questions', 'Cleaned_Question']].head()  # Remplacer 'question' par le nom correct de la colonne

Index(['questions', 'answers'], dtype='object')


,questions,Cleaned_Question
0,What is a novel coronavirus?,what is a novel coronavirus
1,Why is the disease being called coronavirus di...,why is the disease being called coronavirus di...
2,How does the virus spread?,how does the virus spread
3,Can I get COVID-19 from food (including restau...,can i get covid from food including restaurant...
4,Will warm weather stop the outbreak of COVID-19?,will warm weather stop the outbreak of covid


Tokenization et Lemmatisation et elimination de stopwords

In [7]:
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 80.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [8]:
import spacy
import re

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")  # Initialize nlp here

def text_preprocessing(text):
    text = text.lower()  # Conversion en minuscules
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Supprimer tout sauf lettres et espaces
    text = re.sub(r'\s+', ' ', text)  # Remplace les espaces multiples par un seul
    doc = nlp(text)
    tokens = [token.text for token in doc]
    lemmas = [token.lemma_ for token in doc]
    stopwords_removed = [token.lemma_ for token in doc if not token.is_stop]
    text = " ".join(stopwords_removed)
    return text

# Apply the function to the correct columns
df['Cleaned_question'] = df['questions'].apply(text_preprocessing)
df['Cleaned_answer'] = df['answers'].apply(text_preprocessing) # Changed 'answer' to 'answers'

# Vérification après le traitement
print(df[['answers', 'Cleaned_answer']].head()) # Changed 'answer' to 'answers'
df[['questions', 'Cleaned_question']].head()

                                             answers  \
0  A novel coronavirus is a new coronavirus that ...   
1  On February 11, 2020 the World Health Organiza...   
2  The virus that causes COVID-19 is thought to s...   
3  Currently there is no evidence that people can...   
4  It is not yet known whether weather and temper...   

                                      Cleaned_answer  
0  novel coronavirus new coronavirus previously i...  
1  february world health organization announce of...  
2  virus cause covid think spread mainly person p...  
3  currently evidence people covid eat handle foo...  
4  know weather temperature affect spread covid v...  


,questions,Cleaned_question
0,What is a novel coronavirus?,novel coronavirus
1,Why is the disease being called coronavirus di...,disease call coronavirus disease covid
2,How does the virus spread?,virus spread
3,Can I get COVID-19 from food (including restau...,covid food include restaurant takeout produce ...
4,Will warm weather stop the outbreak of COVID-19?,warm weather stop outbreak covid


In [9]:
# Sauvegarder le dataset nettoyé et préparé
df.to_csv('cleaned_chatbot_data.csv', index=False)

Charger les Données avec pandas

In [10]:
# Test

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import spacy
import re

# Charger SpaCy
nlp = spacy.load("en_core_web_sm")

# Nettoyage des textes
#def nettoyer_texte(texte):
#    texte = re.sub(r"[^a-zA-Z0-9\s]", "", texte.lower())
#    return texte.strip()

# Lemmatisation
#def lemmatize_texte(texte):
#    doc = nlp(texte)
#    lemmes = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
#    return " ".join(lemmes)

# dataset
df = pd.read_csv('/content/covid_faq.csv')

# Nettoyage et lemmatisation
#df["Cleaned_Question"] = df["question"].apply(nettoyer_texte)
#df["Lemmatized_Question"] = df["Cleaned_Question"].apply(lemmatize_texte)

# TF-IDF avec des ajustements
vectorizer = TfidfVectorizer(
    stop_words="english",        # Supprime stopwords anglais
    min_df=1,                    # Inclut mots présents dans au moins 1 document
    max_df=0.8,                  # Exclut mots présents dans plus de 80% des documents
    max_features=50             # Limite à 50 mots-clés
)
tfidf_matrix = vectorizer.fit_transform(df["questions"])

# Créer un DataFrame lisible
mots_cles = vectorizer.get_feature_names_out()
documents = [f"Question {i+1}" for i in range(tfidf_matrix.shape[0])]
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), index=documents, columns=mots_cles)

In [11]:
# Affichage des mots-clés et scores
print("Matrice TF-IDF :")
df_tfidf.head()

Matrice TF-IDF :


,19,animals,causes,child,children,cleaning,close,cloth,community,complex,...,surfaces,symptoms,test,tested,time,tracing,united,use,virus,water
Question 1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
Question 2,0.707107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
Question 3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.506911,0.000000
Question 4,0.407130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.817612
Question 5,0.337187,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


In [12]:
df_tfidf.tail()

,19,animals,causes,child,children,cleaning,close,cloth,community,complex,...,surfaces,symptoms,test,tested,time,tracing,united,use,virus,water
Question 113,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
Question 114,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.302292,0.372959,...,0.392959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.287320
Question 115,0.151952,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.321057,0.396111,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.305155
Question 116,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.647492,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
Question 117,0.271536,0.0,0.545308,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.520966,0.000000


Calculer la Similarité Cosinus

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import spacy
import re
from sklearn.metrics.pairwise import cosine_similarity # Import cosine_similarity

# Charger SpaCy
nlp = spacy.load("en_core_web_sm")

# Nettoyage des textes
#def nettoyer_texte(texte):
#    texte = re.sub(r"[^a-zA-Z0-9\s]", "", texte.lower())
#    return texte.strip()

# Lemmatisation
#def lemmatize_texte(texte):
#    doc = nlp(texte)
#    lemmes = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
#    return " ".join(lemmes)

# dataset
df = pd.read_csv('/content/covid_faq.csv')

# Nettoyage et lemmatisation
#df["Cleaned_Question"] = df["question"].apply(nettoyer_texte)
#df["Lemmatized_Question"] = df["Cleaned_Question"].apply(lemmatize_texte)

# TF-IDF avec des ajustements
vectorizer = TfidfVectorizer(
    stop_words="english",        # Supprime stopwords anglais
    min_df=1,                    # Inclut mots présents dans au moins 1 document
    max_df=0.8,                  # Exclut mots présents dans plus de 80% des documents
    max_features=50             # Limite à 50 mots-clés
)
tfidf_matrix = vectorizer.fit_transform(df["questions"])

# Créer un DataFrame lisible
mots_cles = vectorizer.get_feature_names_out()
documents = [f"Question {i+1}" for i in range(tfidf_matrix.shape[0])]
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), index=documents, columns=mots_cles)


def trouver_questions_similaires(user_question):
    # Nettoyer et lemmatiser la question utilisateur
    #user_question_cleaned = nettoyer_texte(user_question)
    #user_question_lemmatized = lemmatize_texte(user_question_cleaned)

    # Transformer la question utilisateur en vecteur TF-IDF
    user_vec = vectorizer.transform([user_question])

    # Calculer la similarité cosinus entre la question utilisateur et les questions du dataset
    similarities = cosine_similarity(user_vec, tfidf_matrix)[0]  # Flatten the array
    print("Similarités :", similarities)

    # Trouver le score maximal
    max_score = similarities.max()

    # Identifier tous les index des questions ayant le score maximal
    best_match_indices = [i for i, score in enumerate(similarities) if score == max_score]

    # Rassembler toutes les questions et réponses correspondantes
    results = []
    for idx in best_match_indices:
        question = df["questions"].iloc[idx]
        answer = df["answers"].iloc[idx]
        results.append((question, answer, max_score))

    return results
    # Exemple de question utilisateur
user_question = "What is a novel coronavirus?"

# Trouver les questions similaires et leurs réponses
resultats_similaires = trouver_questions_similaires(user_question)

# Afficher les résultats
print("Question utilisateur :", user_question)
if resultats_similaires:
    for i, (question_similaire, reponse_similaire, score) in enumerate(resultats_similaires, start=1):
        print(f"\nRésultat {i} :")
        print(f"  Question similaire trouvée : {question_similaire}")
        print(f"  Réponse correspondante : {reponse_similaire}")
        print(f"  Score de similarité : {score}")
else:
    print("Aucune question similaire trouvée.")

Similarités : [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Question utilisateur : What is a novel coronavirus?

Résultat 1 :
  Question similaire trouvée : What is a novel coronavirus?
  Réponse correspondante : A novel coronavirus is a new coronavirus that has not been previously identified. The virus causing coronavirus disease 2019 (COVID-19), is not the same as the coronaviruses that commonly circulate among humans and cause mild illness, like the common cold.
  Score de similarité : 0.0

Résultat 2 :
  Question similaire trouvée : Why is the disease being called coronavirus disease 2019, COVID-19?
  Réponse correspondante : On February 11, 2020 the World Health Organization 

Ajouter un Seuil pour les Questions Hors Dataset

In [14]:
def trouver_questions_similaires(user_question, seuil_similarite):
    # Nettoyer et lemmatiser la question utilisateur
    #user_question_cleaned = nettoyer_texte(user_question)
    #user_question_lemmatized = lemmatize_texte(user_question_cleaned)

    # Transformer la question utilisateur en vecteur TF-IDF
    user_vec = vectorizer.transform([user_question])

    # Calculer la similarité cosinus entre la question utilisateur et les questions du dataset
    similarities = cosine_similarity(user_vec, tfidf_matrix)[0]  # Flatten the array

    # Trouver les index des questions avec une similarité au-dessus du seuil
    indices_valables = [i for i, score in enumerate(similarities) if score >= seuil_similarite]

    # Rassembler toutes les questions et réponses correspondantes au seuil
    results = [
        (df["questions"].iloc[i], df["answers"].iloc[i], similarities[i])
        for i in indices_valables
    ]

    return results
    # Exemple de question utilisateur
user_question = "Is it okay for me ?"
seuil = 0.8

# Trouver les questions similaires et leurs réponses
resultats_similaires = trouver_questions_similaires(user_question, seuil)

# Afficher les résultats
print("Question utilisateur :", user_question)
if resultats_similaires:
    for i, (question_similaire, reponse_similaire, score) in enumerate(resultats_similaires, start=1):
        print(f"\nRésultat {i} :")
        print(f"  Question similaire trouvée : {question_similaire}")
        print(f"  Réponse correspondante : {reponse_similaire}")
        print(f"  Score de similarité : {score}")
else:
    print("Aucune question similaire trouvée.")

Question utilisateur : Is it okay for me ?
Aucune question similaire trouvée.


In [15]:
pip install sentence-transformers

In [16]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Charger le modèle SBERT pré-entraîné
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encoder les questions du dataset et la question utilisateur en vecteurs sémantiques
corpus_embeddings = model.encode(df["questions"].tolist())
user_question = "How can I protect myself?"
user_embedding = model.encode([user_question])

# Calculer la similarité cosinus entre la question utilisateur et le corpus
similarities = cosine_similarity(user_embedding, corpus_embeddings)

# Trouver la question la plus similaire
best_match_idx = similarities.argmax()
best_match_score = similarities[0, best_match_idx]

# Afficher la question et la réponse correspondante
question_similaire = df["questions"].iloc[best_match_idx]
reponse_similaire = df["answers"].iloc[best_match_idx]

# Résultats
print(f"Question similaire trouvée : {question_similaire}")
print(f"Réponse correspondante : {reponse_similaire}")
print(f"Score de similarité : {best_match_score}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Question similaire trouvée : How can I protect myself?
Réponse correspondante : Visit the How to Protect Yourself & Others page to learn about how to protect yourself from respiratory illnesses, like COVID-19.
Score de similarité : 0.9999998211860657


Améliorations possibles

In [17]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Charger le modèle SBERT pré-entraîné
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encoder les questions du dataset et la question utilisateur en vecteurs sémantiques
corpus_embeddings = model.encode(df["questions"].tolist())
user_question = "What should I do if I have had close contact with someone who has COVID-19?"
user_embedding = model.encode([user_question])

# Calculer la similarité cosinus entre la question utilisateur et le corpus
similarities = cosine_similarity(user_embedding, corpus_embeddings)

# Trouver la question la plus similaire
best_match_idx = similarities.argmax()
best_match_score = similarities[0, best_match_idx]

# Afficher la question et la réponse correspondante
question_similaire = df["questions"].iloc[best_match_idx]
reponse_similaire = df["answers"].iloc[best_match_idx]

# Résultats
print(f"Question similaire trouvée : {question_similaire}")
print(f"Réponse correspondante : {reponse_similaire}")
print(f"Score de similarité : {best_match_score}")

Question similaire trouvée : What should I do if I have had close contact with someone who has COVID-19?
Réponse correspondante : Be alert for symptoms. Watch for fever, cough, shortness of breath, or other symptoms of COVID-19. Take your temperature and follow CDC guidance if you have symptoms.
Score de similarité : 1.000000238418579


In [18]:
df['questions']

,questions
0,What is a novel coronavirus?
1,Why is the disease being called coronavirus di...
2,How does the virus spread?
3,Can I get COVID-19 from food (including restau...
4,Will warm weather stop the outbreak of COVID-19?
...,...
112,Can the sun’s ultraviolet (UV) light disinfect...
113,How often do surfaces (such as handrails and i...
114,Do procedures for responding to formed fecal (...
115,Can lifeguards monitor how visitors socially d...


les étapes de création du chatbot

In [ ]:
!pip install streamlit
!pip install pyngrok
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.9 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

# Fonction pour nettoyer le texte
#def nettoyer_texte(texte):
#    if not isinstance(texte, str):
#        return ""
#    texte = re.sub(r"<.*?>", "", texte)  # Supprimer les balises HTML
#    texte = re.sub(r"[^a-zA-Z0-9éèêçàùâû\s]", "", texte)  # Supprimer les caractères spéciaux
#    texte = re.sub(r"\s+", " ", texte)  # Réduire les espaces multiples
#    return texte.strip()

# Charger le dataset MedQuAD (remplacez ceci par votre propre fichier CSV)
df = pd.read_csv("/content/covid_faq.csv")  # Assurez-vous que le fichier CSV est bien chargé dans Colab ou votre environnement

# Nettoyer les questions et réponses
#df["question_cleaned"] = df["question"].apply(nettoyer_texte)
#df["answer_cleaned"] = df["answer"].apply(nettoyer_texte)

# Initialiser et entraîner TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df["questions"])

# Fonction pour trouver une question similaire avec TF-IDF
def trouver_question_similaire(user_question):
    #user_question_cleaned = nettoyer_texte(user_question)
    user_vec = vectorizer.transform([user_question])
    similarites = cosine_similarity(user_vec, tfidf_matrix)
    index_similaire = similarites.argsort()[0, -1]
    return index_similaire, similarites[0][index_similaire]

# Interface utilisateur Streamlit avec un titre modifié
st.title("Chatbot Médical avec TF-IDF et Similarité Cosinus")

user_question = st.text_input("Bonjour! Posez votre question ici :")

if user_question:
    # Trouver la question similaire avec TF-IDF
    index_tfidf, tfidf_score = trouver_question_similaire(user_question)

    # Afficher les résultats
    st.write("Question similaire :", df["questions"].iloc[index_tfidf])
    st.write("Réponse :", df["answers"].iloc[index_tfidf])
    st.write("Score TF-IDF :", tfidf_score)

In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok

# Assurez-vous que votre authtoken est configuré
ngrok.set_auth_token("2qo7M4sn11ETX4HppPxRT7IANq0_6c8ar8wxUap3F9zFm4z7f")

# Lancer le tunnel ngrok sur le port 8501 pour Streamlit
public_url = ngrok.connect(8501)  # Assurez-vous de spécifier le port 8501
print("Streamlit est accessible via :", public_url)
# Lancer Streamlit en arrière-plan
!streamlit run app.py &>/content/logs.txt &